In [2]:
import pandas as pd

In [3]:
# base json file needs to be encased on [] and have a comma separating each item
# modifies the json file to be readable. into business.json

import os.path

def reformatJson():
    with open('data/meta-California.json', 'r') as infile, open('data/business.json', 'w') as outfile:
        outfile.write('[\n')
        lines = infile.readlines()

        # adds comma to every line except last
        for i, line in enumerate(lines):
            if i < len(lines) - 1: line = line.rstrip() + ',\n'
            outfile.write(line)

        outfile.write(']')
        
if not os.path.isfile('data/business.json') : reformatJson()

In [4]:
cali = pd.read_json('data/business.json')

In [5]:
dropped = cali.drop(columns=['gmap_id', 'description', 'MISC', 'state', 'url', 'relative_results'])
dropped = dropped.drop(columns=['price']) # for now
dropped = dropped.dropna()
dropped.columns.tolist()

['name',
 'address',
 'latitude',
 'longitude',
 'category',
 'avg_rating',
 'num_of_reviews',
 'hours']

In [6]:
# price is a factor 
# price_dummy = pd.get_dummies(dropped['price'])

# # category is an subset of 1465 categories
# dropped['category'].explode().unique().size
#     # not sure what to do with that
#         # theres things like 'coffee shop' and 'coffee store'
#     # maybe we can try to reduce these categories somehow
# # hours is an array of an array [day of week, starthr-endhr]

In [7]:
import re

def string_to_hours (string : str, AM = False) -> (float, bool):
    match = re.match(r'(\d+)(?::(\d+))?(\D*)', string)
    hours, minutes, period = match.groups()
    AM = (period == '' and AM) or period == "AM"
    return (float(hours) + (0 if AM else 12) + (float(minutes) if minutes else 0) / 60, AM)

def range_to_hours (string: str) -> float:
    if string == 'Open 24 hours': return 24
    elif string == 'Closed' : return 0
    try:
        start, end = string.split('–')
        endTime, endPeriod = string_to_hours(end)
        startTime = string_to_hours(start, endPeriod)[0]
        difference = endTime - startTime
        return difference if difference > 0 else 24 + difference
    except:
        raise Exception("Couldn't enumerate: " + string)


In [8]:
weekly_hours = dropped['hours'].map(\
    lambda entry: sum(map(\
        lambda day: range_to_hours(day[1]), entry)\
    )\
)

In [9]:
modified = dropped.copy()
modified = modified.drop(columns=['hours'])
modified['weekly_hours'] = weekly_hours
modified.head()

,name,address,latitude,longitude,category,avg_rating,num_of_reviews,weekly_hours
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",34.058092,-118.292130,[Korean restaurant],4.4,18,74.5
2,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",34.023669,-118.232930,[Fabric store],3.3,6,40.0
3,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",34.036694,-118.249421,[Fabric store],4.3,7,40.0
4,Matrix International Textiles,"Matrix International Textiles, 1363 S Bonnie B...",34.015505,-118.181839,[Fabric store],3.5,6,45.0
5,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",33.916402,-118.010855,[Restaurant],4.5,18,63.0


In [35]:
target_city = "Davis"
df = modified
limit_city = df[df.apply(lambda row : len(row['address'].split(', ')) >= 3 and row['address'].split(', ')[2] == target_city, axis=1)]
print(limit_city.size)
limit_city.head()

5072


,name,address,latitude,longitude,category,avg_rating,num_of_reviews,weekly_hours
926,Meadow Ridge Apartments,"Meadow Ridge Apartments, 4447 Cowell Blvd, Dav...",38.548937,-121.697093,[Apartment complex],4.3,17,35.0
1628,Leslie's,"Leslie's, 1321 W Covell Blvd, Davis, CA 95616",38.561492,-121.764304,"[Swimming pool supply store, Swimming pool rep...",3.8,18,69.0
1629,BROS LIQUOR,"BROS LIQUOR, 1046 Olive Dr, Davis, CA 95616",38.541587,-121.737174,"[Liquor store, Convenience store, Grocery store]",5.0,5,65.0
2172,UC Davis Facilities Management: Energy Conserv...,UC Davis Facilities Management: Energy Conserv...,38.536058,-121.750787,[Environmental program],5.0,4,45.0
2173,Carl's Jr.,"Carl's Jr., Student Housing, Davis, CA 95616",38.538825,-121.753217,"[Fast food restaurant, Hamburger restaurant, R...",3.0,7,98.0
